In [2]:
#pip install mplsoccer

     ---------------------------------------- 78.8/78.8 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 7.5/7.5 MB 6.8 MB/s eta 0:00:00
     -------------------------------------- 429.4/429.4 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import json
from mplsoccer.pitch import Pitch, VerticalPitch
import requests

In [57]:
#fill out necessarry information of about player and plot type here

player_id = 453 #undestat player id
is_season = True #if is_season = True, then specify season
season = 2022 #2020 corresponds to 2020/21 
highlight_goals = False

In [40]:
#data scraping 

def get_player_data(player_id):
    base = "https://understat.com/player/"
    base_url = base + str(player_id)
    url = base_url
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    scripts = soup.find_all('script')
    strings = scripts[3].string

    ind_start = strings.index("('") + 2
    ind_end = strings.index("')")

    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')

    data = json.loads(json_data)

    finaldata = pd.DataFrame.from_dict(data)
    return finaldata

In [51]:
finaldata = get_player_data(player_id)

In [52]:
# Data Wrangling

# 수치형 변환

finaldata["X"] = pd.to_numeric(finaldata["X"])
finaldata["Y"] = pd.to_numeric(finaldata["Y"])
finaldata["season"] = pd.to_numeric(finaldata["season"])
finaldata["minute"] = pd.to_numeric(finaldata["minute"])
finaldata["xG"] = pd.to_numeric(finaldata["xG"])

# 라이브러리 구현을 위해 *120과 80을 해줘야 하나 봅니다

finaldata["X"] = finaldata["X"] * 120
finaldata["Y"] = finaldata["Y"] * 80  

In [53]:
# 페넕티 득점 제외
# "result", "X", "Y", "xG", "player", "season" 컬럼만 부름 

finaldata = finaldata[finaldata["situation"] != "Penalty"]
finaldata = finaldata[["result", "X", "Y", "xG", "player", "season"]]

In [59]:
if is_season == True:
        finaldata = finaldata[finaldata["season"] == season]
        
        year = {2014: "2014/15", 2015: "2015/16", 2016: "2016/17", 2017: "2017/18", 2018: "2018/19", 
                2019: "2019/20", 2020: "2020/21", 2021: "2021/22", 2022: "2022/23"}
        
        finaldata["season"] = finaldata["season"].map(year)
        
        szn = finaldata["season"].values[0]
        
        x = 45.2
        y = 62
        
elif is_season == False:
        finaldata["all"] = finaldata["season"]
        finaldata["all"] = "All Seasons"
        szn = finaldata["all"].values[0]
        x = 47
        y = 62

C:\Users\koaro\AppData\Local\Temp\ipykernel_19812\3018150036.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finaldata["season"] = finaldata["season"].map(year)
